### Insurance model deployment in SageMaker

In [ ]:
!pip install sagemaker >> /dev/null
%store -r

In [ ]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = SECRET_KEY
os.environ["AWS_DEFAULT_REGION"] = REGION_NAME
HOME = os.getenv("HOME")
if HOME:
    EXECUTABLE_DIR = os.path.join(HOME,".local", "bin")
    PATH = os.getenv("PATH")
    if EXECUTABLE_DIR not in PATH:
        os.environ["PATH"] = f"{PATH}:{EXECUTABLE_DIR}"
    PATH = os.getenv("PATH")

### Libraries

In [ ]:
import os
import boto3
import re
import time
import json
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess

### Deploy the model in sagemaker

In [ ]:
data_capture_prefix = "{}/datacapture".format(PREFIX)
s3_capture_upload_path = "s3://{}/{}".format(BUCKET, data_capture_prefix)          
reports_prefix = "{}/reports".format(PREFIX)
s3_report_path = "s3://{}/{}".format(BUCKET, reports_prefix)
code_prefix = "{}/code".format(PREFIX)
s3_code_preprocessor_uri = "s3://{}/{}/{}".format(BUCKET, code_prefix, "preprocessor.py")
s3_code_postprocessor_uri = "s3://{}/{}/{}".format(BUCKET, code_prefix, "postprocessor.py")

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))
print("Preproc Code path: {}".format(s3_code_preprocessor_uri))
print("Postproc Code path: {}".format(s3_code_postprocessor_uri))

In [ ]:
bashCommand = "tar -cvpzf model.tar.gz model.joblib"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

#### Upload trained model.joblib file in sagemaker

In [ ]:
model_file = open("model.joblib", "rb")
s3_key = os.path.join(PREFIX, "model.joblib")
boto3.Session().resource("s3").Bucket(BUCKET).Object(s3_key).upload_fileobj(model_file)

In [ ]:
boto_session = boto3.session.Session(aws_access_key_id=ACCESS_KEY,
                                     aws_secret_access_key=SECRET_KEY,
                                     region_name=REGION_NAME)
s3 = boto_session.resource('s3')
#Upload tar.gz to bucket
response = s3.meta.client.upload_file('model.tar.gz', BUCKET, 'model.tar.gz')

In [ ]:
%%writefile inference_script.py
import joblib
import os

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

In [ ]:
from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=f"s3://{BUCKET}/model.tar.gz",
    role=ROLE,
    entry_point='inference_script.py',
    framework_version='0.23-1')

### Deploying...

In [ ]:
ENDPOINT_NAME = "DEMO-insurance-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
%store ENDPOINT_NAME

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

print("ENDPOINT_NAME = {}".format(ENDPOINT_NAME))

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    endpoint_name=ENDPOINT_NAME,
    data_capture_config=data_capture_config,
    wait=False
)
print("Sagemaker will take 10-15 mins to create the endpoint")

In [ ]:
CLEANUP = False
if CLEANUP:
    client = boto3.client(service_name="sagemaker", aws_access_key_id=ACCESS_KEY,
                                     aws_secret_access_key=SECRET_KEY,
                                     region_name=REGION_NAME)
    client.delete_endpoint(EndpointName = ENDPOINT_NAME)